In [ ]:
import scipy.io as sio
import mne

import numpy as np


from mne.decoding import ReceptiveField
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler


from tqdm.auto import tqdm

import random

# location of files
stimuli_location = "Stimuli/Text"
eeg_location = "EEG/Subject"

### Creating the Suprisal vector.

Now that the similarity measures for each word have been calculated, we need to create the actual disimilarity vector. Basicly the similarity values are put into the right place on the vector that corrosponds with the onset of the word. They are also subtracted from 1 to create dismilarity values from similarity values ( both values are between 0-1 )

In [ ]:
# keep things relatively clean looking, comment out for more information.

mne.set_log_level('error')

In [ ]:
# some general variables.

# what subjects to process. (i only included subject 19 data here)
subs =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
# subs =[19]

# the similarity vectors to use (that we calculated above)
# pearson.npy for broderic dissimilarity values
# bert-scores.npy for bert dissimilarity values
#similarity_location = 'vectors/pearson.npy'
vector_loc = 'vectors/'
#vector = 'static'

# similarity_location = 'vectors/bert_2sen_onedir2.npy'

# frequency of the EEG data
sfreq = 128

# number of channels
n_channels = 128

# Define the delays that we will use in the receptive field (0 = event, so this means we take into account upto 0.6seconds after event)
tmin, tmax = 0, .6

# the mean similarity for the broderick method, everything is normalised to this.
norm = 0.33820258789060476

results_folder = 'results/'
second_folder = 'randoms/'

In [ ]:
def saveFunc(loc, run_string , scores_subs, coefs_subs, error_subs, length):
    
    if length > 1:
        np.save( results_folder +  'coefs/' + run_string + '_' + length + '.npy', np.array(coefs_subs))
        np.save( results_folder + 'scores/' + run_string + '_' + length + '.npy', np.array(scores_subs))
        np.save( results_folder + 'errors/' + run_string + '_' + length + '.npy', np.array(error_subs))
       
    else: 
        np.save( results_folder +  'coefs/' + run_string + '.npy', np.array(coefs_subs))
        np.save( results_folder + 'scores/' + run_string + '.npy', np.array(scores_subs))
        np.save( results_folder + 'errors/' + run_string + '.npy', np.array(error_subs))
    

sets the similarity values (1- the value for disimilarity) of each words at the onset of the word, thus creating a disimilarity vector. loads the eeg data to create a dismilarity vector of the same size. Takes the opportunity to save all the eeg data into lists of lists to prevent reloading it later on.

In [ ]:
def load_eeg_and_disimilarlity_vector(sim, method):

    # flatten the similarity values (from 3D list to 1D list)
    flat_sim = [word for sentence in [sentence for run in sim for sentence in run] for word in sentence]
    
    # normalize to the values of the broderick similarity values.
    normalise = norm/np.mean(flat_sim)
    
    
    
    # 
    raw_subs = []
    ms_subs = []
    dis_vector_subs = []

    for sub in tqdm(subs):  # for each subject
        raw = []
        ms = []
        dis_vector = []
        #print('Load eeg & dissimilarity values: subject: ', sub, "/", len(subs), end = "\r", flush=True)

        for i in range(1,21):  # for each run

            # load data
            data = sio.loadmat(eeg_location + str(sub) + "/Subject" + str(sub) + "_Run" + str(i) +  ".mat")
            raw_i = data['eegData'].T
            ms_i = data['mastoids'].T
            loc = stimuli_location + "/Run" + str(i) + ".mat"
            words = sio.loadmat(loc)

            # create list of word similaritys in order and normalise.
            norm_flat_sim = [simz * normalise for sentence in sim[i-1] for simz in sentence]


            if len(norm_flat_sim)!= len(words['onset_time']):
                print(sub,i, len(norm_flat_sim), len(words['onset_time']))

            # create empty vector of same size as the eeg.
            dis_vector_i = np.zeros((1,raw_i.shape[1]))
            
            if method == 'random_loc': 
                random_locations = random.sample(range(0,raw_i.shape[1]), len(norm_flat_sim))
            
            #print(asdfasdfasdfasdf)
            # for each word, find its onset, and then place it there in the empty vector.
            for j in range(len(norm_flat_sim)):
                
                
                if method == 'random_loc':
                    on = random_locations[j]
                else: 
                    on = int(np.floor(words['onset_time'][j] * sfreq))
                    
                dis_vector_i[0][on] = (1-norm_flat_sim[j]) 
            

            
                # This is also where the random vectors and such are created. 
                # comment out to do something else then the normal method


                # dis_vector_i[0][on] = 0.33820258789060476                     # one at onset
                # dis_vector_i[0][on] = random.uniform(0, 1)  # random value on onset

    #         for i in range(1,len(dis_vector_i[0])):          # value at specific or all times
    #             dis_vector_i[0][i] = 1


            raw.append(raw_i)     
            dis_vector.append(dis_vector_i)
            ms.append(ms_i)

        raw_subs.append(raw)
        ms_subs.append(ms)
        dis_vector_subs.append(dis_vector)
        
    print('Load eeg & dissimilarity values: Done')
    return(dis_vector_subs,ms_subs,raw_subs)

    


# Preprocessing

For each run of each subject, load into mne and preprocess. 
preprocessing consists of applying a bandpass filter(1 to 8hz) and rerefrencing to the average of the two mastoids. 

The band pass filter filters out all frequencys outside of the range specified.
referencing is done by averaging the two mastoids and subtracting it from each channel. this way head movements and such can be removed from the eeg.

In [ ]:
def process_eeg(dis_vector, raw_mastoids, raw_eeg, var_limit): 
    
    fixes = 0
    fixes2 = 0
    subject_data = []
    for sub in tqdm(subs):
        #print('Process eeg: subject: ', sub, "/", len(subs), end = "\r", flush=True)
        raws = np.empty((20,), dtype=object)
        ms = np.empty((20,), dtype=object)
        dv = np.empty((20,), dtype=object)
        filterd = np.empty((20,), dtype=object)
        data = {}
        for i in range(1,21):

            # load data of the prev cell
            raws[i-1] = raw_eeg[subs.index(sub)][i-1].T
            ms[i-1] = raw_mastoids[subs.index(sub)][i-1].T
            dv[i-1] = dis_vector[subs.index(sub)][i-1].T


            # add mastoids as the last two channels.
            raw = np.concatenate((raw_eeg[subs.index(sub)][i-1],raw_mastoids[subs.index(sub)][i-1]))
            
            
            montage = mne.channels.make_standard_montage('biosemi128');
            montage.ch_names = montage.ch_names + ['M1', 'M2']
            montage.dig = montage.dig + [montage.dig[4],montage.dig[5]]
            

            info = mne.create_info(montage.ch_names[:130], sfreq, 'eeg', montage=montage)
            raw = mne.io.RawArray(raw, info)
            raw.add_channels


            # remove the standerd reference mne sets upon it.
            raw, _ = mne.set_eeg_reference(raw, [])

            # reference instead using the average of the last two channels (mastoids)
            raw, _ = mne.set_eeg_reference(raw, montage.ch_names[128:130])

            # bandpass filter the data without the mastoids
            #raw.info['bad'] = montage.ch_names[128:130]
            raw = raw.drop_channels(montage.ch_names[128:130])
            raw = raw.filter(1,8)
            
            test = raw.get_data()
        
            var = [np.var(channel[1280:-1280]) for channel in test]
            raw.info['bads'] = np.array(montage.ch_names[:128])[np.array(var[:128])>var_limit]
            fixes2 += raw.info['bads'].size
            if raw.info['bads'].size > 0: fixes +=1
            raw.interpolate_bads()

            
            #print(raw.get_data.size())
            filt = raw.get_data()
            filterd[i-1] = filt[:128].T   


        data['raw'] = raws
        data['ms'] = ms
        data['dv'] = dv
        data['filterd'] = filterd
        subject_data.append(data)
    
    print('Process eeg: Done')
    print("Total fixes: ", fixes)
    print("Total fixes: ", fixes2)
    return(subject_data)


# Learning

learning is done using 5 folds regression, its resulting coeffiecients (weights) are the actual outputs we are looking for. the scores are the prediction accuracy per channel

In [ ]:
def TRF(subject_data, start=0, end=1, save=None, shuffle=False, second_folder = 'randoms/'):
    
    for run in tqdm(range(start,end)):
    
        #print('run', run)
        scores_subs=[]
        coefs_subs=[]
        error_subs=[]
        for sub in tqdm(subs):

            # run value as random seed. makes reproduction possible if we need extra info later on.
            random.seed(run)
            data = subject_data[subs.index(sub)]
            dv = np.vstack(data['dv'])

            if shuffle:
                # create a list of all disimilarity values.
                ddv = np.array([d for d in dv if d>0])

                # shuffle this list.
                random.shuffle(ddv)

                # for each non 0 in real dissimilarity vector
                i = 0
                for j in range(len(dv)):
                    if dv[j] > 0:

                        # add the i element of the shuffled list of dissimilarity values.
                        dv[j] = ddv[i]
                        i+=1

            Y = np.vstack(data['filterd']) 

            # normalisation method. basicly just subtracts the average and devides by the standerd deviations
            scaler = StandardScaler()           # initalise scaler
            scaler.fit(Y)                       # finds the average and sd
            Y = scaler.transform(Y)      # scales each value.


            #Initialize the model
            rf = ReceptiveField(tmin, tmax, sfreq, feature_names=['envelope'],
                                estimator=1., scoring='corrcoef')

            # calculate how many items there are between the delays
            n_delays = int((tmax - tmin) * sfreq) + 2

            # setup 5fold CV
            n_splits = 5
            cv = KFold(n_splits)


            DV = np.array([dv[i-78:i,] for i in range(78,dv.shape[0])])


            # Simple linear regression for each time step using 5fold CV
            coefs = np.zeros((n_splits, n_channels, n_delays))
            scores = np.zeros((n_splits, n_channels))


            err= []
            # Iterate through splits, fit the model, and predict/test on held-out data
            for ii, (train, test) in enumerate(cv.split(dv)):  # split the data 4:1 (train:test) for each different way.
                #print('split %s / %s' % (ii + 1, n_splits))

                rf.fit(dv[train], Y[train])


                scores[ii] = rf.score(dv[test], Y[test])
                # coef_ is shape (n_outputs, n_features, n_delays). we only have 1 feature
                coefs[ii] = rf.coef_[:, 0, :]

                # calculate errors

                W = np.flip(coefs[ii],1) # define Weights as the flipped coefs
                #print(test)

                test=test[test>=78] # remove the first 78 values, as we dont have full data for them
                test=test -78 
                #print(test)
                DV2 = DV[test] # select only the dissimilarity values for the test set
                real = Y[test]


                DVs = np.sum(DV2,1) 
                mask = [s for s in range(0,len(DV2)) if DVs[s]!=0]
                DV2 = DV2[mask] # remove all test cases with 0 dv values
                real = real[mask]


                pred = np.array([W @ s for s in DV2]).squeeze() # calculate predicted value.

                m = np.mean(real,0)
                SStot = np.sum((real-m)**2,0)
                SSreg = np.sum((pred-m)**2,0)
                SSres = np.sum((real-pred)**2,0)
                r2 = 1-(SSres/SStot)
                errors = np.array([SStot,SSreg,SSres,r2])

                err.append(errors)



            times = rf.delays_ / float(rf.sfreq)


            scores_subs.append(scores)
            coefs_subs.append(coefs)
            error_subs.append(err)
        
        if shuffle==True and save and ((end-start)>1):
            # save results
            saveFunc(loc, run_string , scores_subs, coefs_subs, error_subs, length)

    
    return(scores_subs, coefs_subs, error_subs) # save last run results

In [ ]:
def train(vector, preprocess = 100000, backup = 'pearson', start=0, end=1, save=True, shuffle=False):
    
    if vector == 'random_loc':
        similarity_location = vector_loc +  backup + '.npy'
    else:
        similarity_location = vector_loc +  vector + '.npy'
    
    # loads the similarity values 
    sim = np.load(similarity_location, allow_pickle=True)
    
    dis_vector, raw_mastoids, raw_eeg = load_eeg_and_disimilarlity_vector(sim, vector)
    data = process_eeg(dis_vector, raw_mastoids, raw_eeg, preprocess)
    
    scores_subs, coefs_subs, error_subs = TRF(data)
    #scores_subs, coefs_subs, error_subs = TRF(data, start=3, end=100, save=save, shuffle=True, second_folder)
    
    
    if end-start > 1:
        combine_multi_runs()
    
    if save:
        if vector == 'pearson':
            run_string = vector + '_' + str(preprocess)
            
        elif vector[:4] == 'bert':
            run_string = vector + '_' + str(preprocess)
            
        elif vector == 'static':
            run_string = vector + '_' + str(preprocess)
        
        elif vector == 'random_loc':
            run_string = vector + '(' + backup + ")" + '_' + str(preprocess)
        
        elif shuffle:
            run_string = 'shuffle(' + vector + ')' + '_' + str(preprocess)
        
        saveFunc(results_folder, run_string, scores_subs, coefs_subs, error_subs, end-start)
            
    
    
    return(scores_subs, coefs_subs, error_subs)
    
#scores_subz, coefs_subz, error_subz = train(vector_loc, method,  999999999999999999)

Below are the commands to create the main models used in the thesis.

In [ ]:
# scores_subz, coefs_subz, error_subz = train('static', 100000) # static_100000.npy
# scores_subz, coefs_subz, error_subz = train('pearson', 100000,save=False) # pearson_100000.npy
# scores_subz, coefs_subz, error_subz = train('random_loc', 100000, backup = 'pearson') # random_loc_100000.npy

#  

# scores_subz, coefs_subz, error_subz = train('bert_0_False', 100000) # bert_1_False_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_1_False', 100000) # bert_1_False_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_2_False', 100000) # bert_1_False_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_3_False', 100000) # bert_1_False_100000.npy
#scores_subz, coefs_subz, error_subz = train('bert_4_False', 100000) # bert_1_False_100000.npy




# scores_subz, coefs_subz, error_subz = train('bert_0_True', 100000) # bert_1_True_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_1_True', 100000) # bert_1_True_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_2_True', 100000) # bert_1_True_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_3_True', 100000) # bert_1_True_100000.npy
# scores_subz, coefs_subz, error_subz = train('bert_4_True', 100000) # bert_1_True_100000.npy